## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json
import os
from subprocess import Popen, PIPE, STDOUT

from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
from haystack.retriever.sparse import ElasticsearchRetriever
from haystack.pipeline import ExtractiveQAPipeline

03/04/2021 20:34:00 - INFO - faiss.loader -   Loading faiss with AVX2 support.
03/04/2021 20:34:00 - INFO - faiss.loader -   Loading faiss.
03/04/2021 20:34:01 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


## Load data

There are four types of dataset associated with AmazonQA:

* Products: combination of Amazon reviews and questions
* QA pairs: heuristics applied to the products dataset to generate QA pairs and query-relevant review snippets (the main contribution from the paper)
* SQuAD style: conversion of QA pairs to extractive QA format
* MS MARCO: conversion of QA pairs to abstractive QA format'

Task:

> Given a set of product reviews and a question concerning a specific product, generate an informative
natural language answer.

So could build a system where you search for a product and then ask questions about that product. Will need a way to lookup Amazon Standard Identification Number (ASIN) per product to be human readable. 

In [ ]:
data = Path('./data/amazon-qa')
!ls {data}

meta_Computers.json.gz	train-qar_clean.jsonl	 train-qar_products.jsonl
train-qar.jsonl		train-qar_meta.jsonl	 train-qar_squad.jsonl
train-qar.jsonl.bak	train-qar_msmarco.jsonl


### Products

In [ ]:
products_df = pd.read_json(data/'train-qar_products.jsonl', lines=True, nrows=10)
products_df.head()

,asin,questions,reviews,category
0,B007F357HQ,"[{'questionText': 'I had shoulder surgery 6 months ago and have a 4"" wide sc...","[{'helpful': [1, 1], 'reviewText': 'Love this - wasn't sure I would as I tho...",Beauty
1,B00CRAJZFW,"[{'questionText': 'is it for iphones', 'questionType': 'yesno', 'answers': [...","[{'helpful': [2, 4], 'reviewText': 'This product arrived exactly as pictured...",Cell_Phones_and_Accessories
2,B002ZAZ7H4,"[{'questionText': 'what is the width and ht of the cells?', 'questionType': ...","[{'helpful': [0, 0], 'reviewText': 'Very well constructed and designed.I lik...",Home_and_Kitchen
3,B008SCP8UE,"[{'questionText': 'is it big for a bunny', 'questionType': 'yesno', 'answers...","[{'helpful': [0, 0], 'reviewText': 'Absolutely love!!! It was cute and easy ...",Pet_Supplies
4,B001O4F8Y4,[{'questionText': 'We are having a problem with the range. We have an old ho...,"[{'helpful': [0, 0], 'reviewText': 'We have a two story home and notice that...",Tools_and_Home_Improvement


### QA pairs

In [ ]:
qar_df = pd.read_json(data/'train-qar.jsonl', lines=True, nrows=10)
qar_df.head()

,asin,category,questionText,questionType,review_snippets,answers,is_answerable,qid
0,B000MP20BU,Toys_and_Games,"Many have stated similar to the following: ""Paint Chips Off Easily; Pieces a...",descriptive,[A lot of reviewers have said things about this puzzle not being that durabl...,[{'answerText': 'The paint has held up through two toddlers and still going ...,1,0
1,B00BOXZZU2,Health_and_Personal_Care,Will these work with the Phillips sonicare handles?,descriptive,[I didn't even realize such a small electric tooth-brush existed till I acci...,"[{'answerText': 'The answer unfortunately, is no. The Slim Sonic is a compac...",0,1
2,B00CSYD4M2,Cell_Phones_and_Accessories,What kind of sim card it use?,descriptive,[I bought this phone a few weeks ago.I am using it in Costa Rica with a Kolb...,"[{'answerText': 'This phone is an unlocked GSM device, it requires a MINI SI...",1,2
3,B00C5TNSRG,Home_and_Kitchen,does anyone know if this dinnerware set does not contain lead or traces of l...,descriptive,[I love my new dishes! They are so versatile. I can set a casual table and y...,[{'answerText': 'According to the internet search: three-layer glass lamin...,0,3
4,B0099XQBD4,Musical_Instruments,I'm thinking of getting in to modular synthesizers. Would this work for that?,descriptive,[Will order another in the near future and arrived very quickly. Easy to ins...,"[{'answerText': 'Yes it will.', 'answerType': 'NA', 'helpful': [1, 1]}, {'an...",0,4


### SQuAD

In [ ]:
squad_df = pd.read_json(data/'train-qar_squad.jsonl', lines=True, nrows=10)
squad_df.head()

,context,qas
0,This is the perfect kit to get started. Everything is miniaturized and comes...,"[{'id': 331392, 'is_impossible': False, 'question': 'What exactly comes in t..."
1,"... it doesn't last quite as long as advertised so therefore, I had to conti...","[{'id': 684949, 'is_impossible': False, 'question': 'How do you apply this p..."
2,This is a pretty cool filter. If you spin it around it will totally change t...,"[{'id': 604553, 'is_impossible': False, 'question': 'Does this come with a c..."
3,This product was the exact match of the original manufactured equipment.. On...,"[{'id': 341653, 'is_impossible': False, 'question': 'How to remove midgate r..."
4,Nice kit. Works well. Adjustable for proper sighting. Good quality. Instruct...,"[{'id': 192046, 'is_impossible': False, 'question': 'does this include the m..."


### MARCO

In [ ]:
marco_df = pd.read_json(data/'train-qar_msmarco.jsonl', lines=True, nrows=10)
marco_df.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[If your VHS tapes are getting old you may have problems like that. Also He...,"[{'is_selected': 1, 'url': '', 'passage_text': 'This product arrived in a &#...",I am having issues with dropped video. Audio is fine. I'm getting video abou...,384160,DESCRIPTION,[]
1,"[No they are not made in USA not the ones I got sorry, I am not sure. I bel...","[{'is_selected': 1, 'url': '', 'passage_text': 'My wife and I looked for mon...",Are all mattress materials made in the USA?,282084,DESCRIPTION,[]
2,[It's not really suited for handling large debris. It should only be used fo...,"[{'is_selected': 1, 'url': '', 'passage_text': 'We purchased our home a few ...",How well does it handle leaves? especially large leaves? I had a KK severa...,454427,DESCRIPTION,[]
3,"[There is no adapter, just the micro SD card., It comes with a standard SD c...","[{'is_selected': 1, 'url': '', 'passage_text': 'I purchased this card for my...",Does this card come with the adapter for the larger slots as in a RaspberryPI?,193420,DESCRIPTION,[]
4,[I have done that to another grease gun before. You would probably have to ...,"[{'is_selected': 1, 'url': '', 'passage_text': 'The first one I got didn't w...",would this work with oil instead of grease? I have a old milling machine wi...,661529,YESNO,[]


### Metadata
Let's pick out the mapping from ASIN to QID from the training set:

In [ ]:
rows = []
with open(data/'train-qar.jsonl', 'r') as f:
    for _, line in tqdm(enumerate(f)):
        row = json.loads(line)
        rows.append((row['asin'], row['category'], row['qid'], row['is_answerable']))

738776it [00:19, 38838.14it/s]


In [ ]:
meta_df = pd.DataFrame(rows, columns=['asin', 'category', 'qid', 'is_answerable'])
meta_df.head()

,asin,category,qid,is_answerable
0,B000MP20BU,Toys_and_Games,0,1
1,B00BOXZZU2,Health_and_Personal_Care,1,0
2,B00CSYD4M2,Cell_Phones_and_Accessories,2,1
3,B00C5TNSRG,Home_and_Kitchen,3,0
4,B0099XQBD4,Musical_Instruments,4,0


In [ ]:
meta_df.shape

(738776, 4)

In [ ]:
assert meta_df['qid'].nunique() == len(meta_df)

In [ ]:
meta_df['category'].value_counts()

Electronics                    169764
Home_and_Kitchen               107423
Sports_and_Outdoors             70824
Tools_and_Home_Improvement      62995
Health_and_Personal_Care        47589
Automotive                      45892
Cell_Phones_and_Accessories     42211
Patio_Lawn_and_Garden           36693
Toys_and_Games                  30838
Office_Products                 26086
Beauty                          24956
Pet_Supplies                    21668
Baby                            14427
Musical_Instruments             14285
Grocery_and_Gourmet_Food        11553
Video_Games                      5901
Clothing_Shoes_and_Jewelry       5671
Name: category, dtype: int64

In [ ]:
meta_df.groupby('category')['asin'].nunique().sort_values(ascending=False)

category
Electronics                    28696
Home_and_Kitchen               17640
Sports_and_Outdoors            11881
Tools_and_Home_Improvement     10283
Automotive                      8172
Health_and_Personal_Care        8051
Cell_Phones_and_Accessories     7133
Patio_Lawn_and_Garden           6108
Toys_and_Games                  5725
Beauty                          4450
Office_Products                 4339
Pet_Supplies                    3349
Baby                            2466
Musical_Instruments             2366
Grocery_and_Gourmet_Food        2272
Clothing_Shoes_and_Jewelry      1021
Video_Games                      886
Name: asin, dtype: int64

In [ ]:
meta_df.groupby('category')['is_answerable'].value_counts()

category                     is_answerable
Automotive                   0                 23656
                             1                 22236
Baby                         1                 10255
                             0                  4172
Beauty                       1                 15243
                             0                  9713
Cell_Phones_and_Accessories  1                 27446
                             0                 14765
Clothing_Shoes_and_Jewelry   1                  3315
                             0                  2356
Electronics                  1                108614
                             0                 61150
Grocery_and_Gourmet_Food     1                  6774
                             0                  4779
Health_and_Personal_Care     1                 29539
                             0                 18050
Home_and_Kitchen             1                 66384
                             0                 41039
Mus

In [ ]:
meta_df.query("category == 'Musical_Instruments'")

,asin,category,qid,is_answerable
4,B0099XQBD4,Musical_Instruments,4,0
55,B00F9ECDRU,Musical_Instruments,55,0
59,B0083FTVB8,Musical_Instruments,59,0
231,B005ETZ7NW,Musical_Instruments,231,0
269,B000EJTXZU,Musical_Instruments,269,0
...,...,...,...,...
738436,B001KPWU7A,Musical_Instruments,738436,0
738527,B00AMPDYDS,Musical_Instruments,738527,0
738581,B005IQGKX2,Musical_Instruments,738581,1
738652,B001KPWU7A,Musical_Instruments,738652,1


## Warmup: no fine-tuning

Let's pick a single category like `Musical_Instruments` and build a `DataFrame` that has `asin`, `context` columns that we can use to create a simple QA system with an existing model fine-tuned on SQuAD:

In [ ]:
qid2category = pd.Series(meta_df["category"].values, index=meta_df["qid"]).to_dict()
qid2category[0]

'Toys_and_Games'

In [ ]:
qid2asin = pd.Series(meta_df["asin"].values, index=meta_df["qid"]).to_dict()
qid2asin[0]

'B000MP20BU'

It seems that all SQuAD entries are answerable (does this make sense?). What about SQuAD v2 with impossible questions?

In [ ]:
qid2isanswer = pd.Series(meta_df["is_answerable"].values, index=meta_df["qid"]).to_dict()
qid2isanswer[4]

0

In [ ]:
rows = []

with open(data/'train-qar_squad.jsonl', 'r', encoding='utf-8') as f:
    for _, line in tqdm(enumerate(f)):
        row = json.loads(line)
        qid = row["qas"][0]["id"]
        if qid2category[qid] == "Musical_Instruments":
            rows.append((qid2asin[qid], row["context"], row["qas"], qid2isanswer[qid]))

455931it [00:10, 41691.95it/s]


In [ ]:
qa_df = pd.DataFrame(rows, columns=['asin', 'text', "qas", 'is_answerable'])
qa_df.head()

,asin,text,qas,is_answerable
0,B005OZE9SA,Works perfectly and easy to use. Software download also great.The only surpr...,"[{'id': 943, 'is_impossible': False, 'question': 'ipad', 'answers': [{'answe...",1
1,B001RR9BZA,I'm not totally happy with it because it squeals a lot and doesn't really he...,"[{'id': 6381, 'is_impossible': False, 'question': 'Does it amplify your voic...",1
2,B00B9060X6,I've tried computer studios but prefer twiddling knobs. With my Portastudio ...,"[{'id': 440217, 'is_impossible': False, 'question': 'Can you record a drum m...",1
3,B009VDW4OW,I just received this drum in the mail. I had no idea how much assembly was r...,"[{'id': 693739, 'is_impossible': False, 'question': 'What are thoughts on di...",1
4,B004STXY3E,This DMX controller is a great start for getting into DMX lighting control. ...,"[{'id': 27683, 'is_impossible': False, 'question': 'I have RGB par cans and ...",1


In [ ]:
qa_df['is_answerable'].value_counts()

1    8694
Name: is_answerable, dtype: int64

In [ ]:
qa_df.shape

(8694, 3)

### Boot ES

In [ ]:
! wget -nc https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [ ]:
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

03/04/2021 20:36:14 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.093s]
03/04/2021 20:36:14 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.012s]
03/04/2021 20:36:14 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.005s]
03/04/2021 20:36:14 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.022s]
03/04/2021 20:36:14 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.003s]


In [ ]:
document_store.delete_all_documents()

03/04/2021 21:20:03 - INFO - elasticsearch -   POST http://localhost:9200/document/_delete_by_query [status:200 request:0.970s]


### Index docs

In [ ]:
docs = [{"text": row["text"], "meta":{"asin": row["asin"], "is_answerable": row["is_answerable"]}} for _, row in qa_df.iterrows()]
docs[0]

{'text': "Works perfectly and easy to use. Software download also great.The only surprise was that the one I ordered, (USB) doesn't work with an iPad.I was thinking it would work with both PC and iPad. My mistake. I use this with Logic Pro X on an iMac running Mavericks (it's replacing an Mbox) and with a Sony Vaio running Windows 7 and get excellent results (don't forget to install the Windows drivers or you'll run into latency issues). I also use it with the Auria App on my iPad Air. I did appreciate the direct line in switch...I could hear exactly what was being played into the unit without having to route through the computer. That was a nice feature. More recently, I was very happy to get this working with my ipad mini. I did purchase a recommended usb powered hub Belkin model &#34; F4U020&#34; and with that - I'm good to play music into and out of my ipad. Focusrite. An industry standard.I bought this specifically for use with an iPad to do mobile recording. The app I use is Auri

In [ ]:
document_store.write_documents(docs)

03/04/2021 21:20:19 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.093s]
03/04/2021 21:20:21 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.437s]
03/04/2021 21:20:22 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.527s]
03/04/2021 21:20:24 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.488s]
03/04/2021 21:20:25 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.512s]
03/04/2021 21:20:27 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.446s]
03/04/2021 21:20:28 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.502s]
03/04/2021 21:20:30 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.486s]


### Retriever

In [ ]:
retriever = ElasticsearchRetriever(document_store=document_store)

### Reader

In [ ]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True, context_window_size=500)

03/04/2021 20:36:44 - INFO - farm.utils -   Using device: CUDA 
03/04/2021 20:36:44 - INFO - farm.utils -   Number of GPUs: 1
03/04/2021 20:36:44 - INFO - farm.utils -   Distributed Training: False
03/04/2021 20:36:44 - INFO - farm.utils -   Automatic Mixed Precision: None
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
03/04/2021 20:36:57 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
03/04/2021 20:36:57 - INFO - farm.utils -   Using device: CUDA 
03/04/2021 20:36:57 - INFO - farm.utils -   Number of GPUs: 1
03/04/2021 20:36:57 - INFO - farm.utils -   Distributed Training: False
03/04/2021 20:36:57 - INFO - farm.utils -   Automatic Mixed Precision: None
03/04/2021 

In [ ]:
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
query = "Is a snare included?"
# drum box
asin = "B009VDW4OW"
number_of_answers_to_fetch = 3

prediction = pipe.run(query=query, filters={"asin": [asin]}, top_k_retriever=10, top_k_reader=number_of_answers_to_fetch)
print(f"Question: {prediction['query']}")
print("\n")
for i in range(number_of_answers_to_fetch):
    print(f"#{i+1}")
    print(f"Answer: {prediction['answers'][i]['answer']}")
    print(f"ASIN: {prediction['answers'][i]['meta']['asin']}")
    print(f"Is answerable?: {prediction['answers'][i]['meta']['is_answerable']}")
    print(f"Context: {prediction['answers'][i]['context']}")
    print('\n\n')

03/04/2021 21:23:18 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.009s]
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 16.05 Batches/s]

Question: Is a snare included?


#1
Answer: this one only came with one
ASIN: B009VDW4OW
Is answerable?: 1
Context: the correct sounds out of it. When I slapped the "bass", it would play a "snare" sound combined with the bass. When I slapped the "snare", I would just get a wood sound.I've also seen images that most cajons come with multiple snares... this one only came with one.I'm really not sure what else to say. I wanted a Cajon to play with.but didn't want to pay 100.00 plus. This was a great option, Easy to put together with the limited tools I had on hand. And cheap enough that I wasn't worried to have 



#2
Answer: If you are a real percussionist, before you build this you may want to get some snare wires
ASIN: B009VDW4OW
Is answerable?: 1
Context: this for me. He said it was super easy and did not require a lot of tools. I've stained it black and added silver glitter paint and it looks snazzy. My only complaint is that it has a minimal snare sound. If you are a real percussion

## Fine-tuning

TODO: convert to real SQuAD format!

In [ ]:
squad = {"data": [{"title": row["asin"], "paragraphs":{"asin": row["asin"], "is_answerable": row["is_answerable"]}} for _, row in qa_df.iterrows()]


In [ ]:
squad_data = {}



In [ ]:
qa_df.iloc[0]["qas"]

[{'id': 943,
  'is_impossible': False,
  'question': 'ipad',
  'answers': [{'answer_start': 1980, 'text': 'for recording. this is a'},
   {'answer_start': 1975, 'text': 'used for recording. this is a'},
   {'answer_start': 1980, 'text': 'for recording. this is a must'},
   {'answer_start': 1984, 'text': 'recording. this is a must'},
   {'answer_start': 1995, 'text': 'this is a must but'}],
  'human_answers': ['No this is not an iPad',
   'Yes. You need a powered USB hub and an iPad camera connection kit. This is NOT officially supported by Focusrite. If you need a DAW program for iPad, Twisted Wave is a great one.',
   'It is not comparable with the IPad...',
   'It is not comparable with the IPad...']}]

In [ ]:
qa_df.query("asin == 'B005OZE9SA'")

,asin,text,qas,is_answerable
0,B005OZE9SA,Works perfectly and easy to use. Software download also great.The only surpr...,"[{'id': 943, 'is_impossible': False, 'question': 'ipad', 'answers': [{'answe...",1
1431,B005OZE9SA,"As for me it works well with my guitar and passive bass, but my active bass ...","[{'id': 70087, 'is_impossible': False, 'question': 'Can I plug a bass guitar...",1
2348,B005OZE9SA,Audio quality is good at the price range. One significant gripe: If you plug...,"[{'id': 216671, 'is_impossible': False, 'question': 'does this come with the...",1
3754,B005OZE9SA,"Actually Mackie makes a competitor as well, the Onyx Blackjack--but the revi...","[{'id': 14592, 'is_impossible': False, 'question': 'I want to get this devic...",1
4955,B005OZE9SA,"great interface, and awesome customer support. had some issues when switchin...","[{'id': 375866, 'is_impossible': False, 'question': 'Is it compatible with W...",1
6048,B005OZE9SA,Guess what... This one is not even working. The 48v light turns on when you ...,"[{'id': 43453, 'is_impossible': False, 'question': 'So. I want to hook up a ...",1
7262,B005OZE9SA,Great little device with well-written drivers. 2 in 2 out is all you really ...,"[{'id': 624651, 'is_impossible': False, 'question': 'I'm a bit confused abou...",1
7292,B005OZE9SA,"Awesome! It hooked right up, no worries, great for a mac! I am using this wi...","[{'id': 275668, 'is_impossible': False, 'question': 'mac', 'answers': [{'ans...",1
7458,B005OZE9SA,This is great for recording at home. I used it to record a 5' Grand Piano an...,"[{'id': 255964, 'is_impossible': False, 'question': 'Can i record myself at ...",1


In [ ]:
qa_df.rename(columns={"text":"context"})[["context", "qas"]].to_json(data/'train-qar_squad-music.jsonl', orient='records', lines=True)

In [ ]:
train_data = "data/amazon-qa/"
reader.train(data_dir=train_data, train_filename="train-qar_squad-music.jsonl", use_gpu=True, n_epochs=1, save_dir="models/haystack")

03/04/2021 21:35:27 - INFO - farm.utils -   Using device: CUDA 
03/04/2021 21:35:27 - INFO - farm.utils -   Number of GPUs: 1
03/04/2021 21:35:27 - INFO - farm.utils -   Distributed Training: False
03/04/2021 21:35:27 - INFO - farm.utils -   Automatic Mixed Precision: None


JSONDecodeError: Extra data: line 2 column 1 (char 3167)